<h1 align="center"> Automotive Electronics Assignment </h1>

<h2 align="center"> Project Lane Detection </h2>

<h3 align="right"> SathiyaNarayanan P<br>17BEC167</h3>

### Project Description:
   This Project is based on the Opencv python module.Identifying the lanes in the roads is common task.This is critical task for automonous vehicle to perform.Here This project based on the simple Lane detection pipeline with computer vision technique.For more info check  [GitHub Link](https://github.com/sathiya2000/New-Project.git)

## Import Packages

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import cv2
from module import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

## Read in an Image

In [ ]:
image = mpimg.imread('images/solidWhiteRight.jpg')

print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)

## Test Image

In [ ]:
PATH = "images"
test_imgs = os.listdir(PATH)
test_imgs

## Color Selection

In [ ]:
img_list = [read_img(PATH +"/" + name) for name in test_imgs]
display_imgs(img_list, test_imgs)
display_imgs(img_list, test_imgs)
save_imgs(img_list, test_imgs, prefix= "Original")

In [ ]:
diff_color_spaces_imgs = []
diff_color_spaces_labels = []
for i in range(len(img_list)):
    diff_color_spaces_imgs.append(img_list[i])
    diff_color_spaces_labels.append("RGB: " +test_imgs[i])
    diff_color_spaces_imgs.append(to_hsv(img_list[i]))
    diff_color_spaces_labels.append("HSV: " +test_imgs[i])
    diff_color_spaces_imgs.append(to_hls(img_list[i]))
    diff_color_spaces_labels.append("HLS: " +test_imgs[i])
    
display_imgs(diff_color_spaces_imgs, diff_color_spaces_labels, cols=3)

In [ ]:
original_imgs = np.copy(img_list)

## Make original image gray

In [ ]:
gray_imgs = [grayscale(img) for img in img_list]
display_imgs(gray_imgs, test_imgs)
save_imgs(gray_imgs, test_imgs, prefix="Gray")

## Darker image

In [ ]:
darkened_imgs = [adjust_gamma(img, 0.5) for img in gray_imgs]
display_imgs(darkened_imgs, test_imgs)
save_imgs(darkened_imgs, test_imgs, prefix="Darkened")

## RGB Color space - color Selection

In [ ]:
white_masks = [isolate_color_mask(img, np.array([210, 210, 210], dtype=np.uint8), np.array([255, 255, 255], dtype=np.uint8)) for img in img_list]
yellow_masks = [isolate_color_mask(img, np.array([190, 190, 0], dtype=np.uint8), np.array([255, 255, 255], dtype=np.uint8)) for img in img_list]
masked_imgs = []
for i in range(len(img_list)):
    mask = cv2.bitwise_or(white_masks[i], yellow_masks[i])
    masked_imgs.append(cv2.bitwise_and(darkened_imgs[i], darkened_imgs[i], mask=mask))

display_imgs(masked_imgs, test_imgs)

## HLS Color Space - Color selection

In [ ]:
white_masks = []
yellow_masks = []
white_masks = [isolate_color_mask(to_hls(img), np.array([0, 200, 0], dtype=np.uint8), np.array([200, 255, 255], dtype=np.uint8)) for img in original_imgs]
yellow_masks = [isolate_color_mask(to_hls(img), np.array([10, 0, 100], dtype=np.uint8), np.array([40, 255, 255], dtype=np.uint8)) for img in original_imgs]
masked_imgs = []
for i in range(len(original_imgs)):
    mask = cv2.bitwise_or(white_masks[i], yellow_masks[i])
    masked_imgs.append(cv2.bitwise_and(darkened_imgs[i], darkened_imgs[i], mask=mask))

display_imgs(masked_imgs, test_imgs)
save_imgs(masked_imgs, test_imgs, prefix='Color_Mask')

## Gaussian Blur

In [ ]:
blurred_imgs = [gaussian_blur(img, kernel_size=7) for img in masked_imgs]
display_imgs(blurred_imgs, test_imgs)
save_imgs(blurred_imgs, test_imgs, prefix='Gaussian')

## Canny Edge Detector

In [ ]:
canny_imgs = [canny(img, low_threshold=70, high_threshold=140) for img in blurred_imgs]
display_imgs(canny_imgs, test_imgs)
save_imgs(canny_imgs, test_imgs, prefix='Canny')

## Area of interest

In [ ]:
aoi_imgs = []

for img in canny_imgs:        
    aoi_imgs.append(get_aoi(img))
    
display_imgs(aoi_imgs, test_imgs)
save_imgs(aoi_imgs, test_imgs, prefix='AOI')

## Hough Transform Line Detection

In [ ]:
hough_lines_imgs = []
lines = []
for img in aoi_imgs:
    lines.append(get_hough_lines(img))
    
for img, line in zip(original_imgs, lines):
    hough_lines_imgs.append(draw_lines(img, line))
    
display_imgs(hough_lines_imgs, test_imgs)
save_imgs(hough_lines_imgs, test_imgs, prefix='Hough')

## Extrapolating Lines

In [ ]:
lane_lines_imgs =[]

for img, line in zip(original_imgs, lines):
    left_lane, right_lane = get_lane_lines(img, line)
    lane_lines_imgs.append(draw_weighted_lines(img, [left_lane, right_lane], thickness= 15))
    
display_imgs(lane_lines_imgs, test_imgs)
save_imgs(lane_lines_imgs, test_imgs, prefix='Extrapolation')